In [1]:
import sys

import keras_tuner as kt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers

from src.cross_validator import KerasTunerAllChannelsCrossValidator
from src.dataset import load_dataset_train_val_all_channels, \
    load_dataset_train_test_all_channels
from src.models import mlp_builder as bare_model_builder
from src.network_utils import count_params

PWD = '../../..'
sys.path.append(PWD)

tf.get_logger().setLevel('ERROR')

In [2]:
PROJECT_NAME = 'mlp'

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_all/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_all/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 500
BATCH_SIZE = 8096
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_all, _, y_all, _ = load_dataset_train_test_all_channels(PWD)

X_all[(2, 11)].shape, y_all[(2, 11)].shape

((22134, 24), (22134,))

In [4]:
X_train, X_val, y_train, y_val = load_dataset_train_val_all_channels(PWD)

X_train = np.concatenate(list(X_train.values()))
y_train = np.concatenate(list(y_train.values()))
X_val = np.concatenate(list(X_val.values()))
y_val = np.concatenate(list(y_val.values()))

rng = np.random.default_rng(seed=42)

train_idx, val_idx = np.arange(len(X_train)), np.arange(len(X_val))
rng.shuffle(train_idx)
rng.shuffle(val_idx)

X_train, y_train = X_train[train_idx], y_train[train_idx]
X_val, y_val = X_val[val_idx], y_val[val_idx]

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((84824, 24), (84824,), (21208, 24), (21208,))

# Model

In [5]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=1, max_value=8, step=1, default=2)
    hp_units_mult = hp.Choice("units_mult", values=[1, 2, 4, 8, 16, 32], default=4)
    hp_unit_decrease_factor = hp.Choice("unit_decrease_factor", values=[1.0, 1.5, 2.0], default=1.5)
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_dropout = hp.Choice("dropout", values=[0.0, 0.2, 0.5])
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_n_hidden_layers, hp_units_mult, hp_unit_decrease_factor, hp_batch_normalization,
                               hp_input_batch_normalization, hp_dropout, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [6]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 18)                450       
                                                                 
 dense_1 (Dense)             (None, 12)                228       
                                                                 
 dense_2 (Dense)             (None, 1)                 13        
                                                                 
Total params: 691
Trainable params: 691
Non-trainable params: 0
_________________________________________________________________


In [7]:
model_callbacks = [callbacks.EarlyStopping(patience=30, min_delta=ES_MIN_DELTA),
                   callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=5)]

# Bayesian tuner

In [8]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

Trial 40 Complete [00h 00m 27s]
val_loss: 8.947473049163818

Best val_loss So Far: 8.628534317016602
Total elapsed time: 00h 54m 53s


# Results

In [9]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_hidden_layers': 3, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 21217
========== Model 1 ==========
{'n_hidden_layers': 7, 'units_mult': 8, 'unit_decrease_factor': 2.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 1624705
========== Model 2 ==========
{'n_hidden_layers': 8, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 67777
========== Model 3 ==========
{'n_hidden_layers': 6, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 51457
========== Model 4 ==========
{'n_hidden_layers': 6, 'uni

# Cross-validation for top 5 models

In [10]:
cross_validator = KerasTunerAllChannelsCrossValidator(bayesian_tuner, list(X_all.values()), list(y_all.values()),
                                                      model_builder, directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                                      n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                                      es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV,
                                                      n_executions=CROSSVAL_N_EXEC, overwrite=OVERWRITE)
model_scores = cross_validator()

{'n_hidden_layers': 3, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 21217
Got score: 8.5943 (8.5749, 8.6136)
Got score: 8.2490 (8.2517, 8.2463)
Got score: 8.3604 (8.3970, 8.3239)
Got score: 8.2219 (8.2524, 8.1913)
Got score: 8.4561 (8.4605, 8.4517)


{'n_hidden_layers': 7, 'units_mult': 8, 'unit_decrease_factor': 2.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 1624705
Got score: 8.5238 (8.7027, 8.3449)
Got score: 8.3433 (8.4816, 8.2049)
Got score: 8.2557 (8.0563, 8.4551)
Got score: 7.9325 (7.9485, 7.9164)
Got score: 8.5758 (8.5155, 8.6361)


{'n_hidden_layers': 8, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 67777
Got score: 8.6234 (8.6394, 8.6074)
Got score: 8.3130 (8.3106, 8.3155)
Got score: 8.3997 (8.3896, 8.4097)
Got score: 8.2124 (8.2041, 8.2208)
Got score: 8.5155 (8.4908, 8.5401)


{'n_hidden_layers': 6, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 51457
Got score: 8.6749 (8.6632, 8.6866)
Got score: 8.4545 (8.4335, 8.4755)
Got score: 8.6952 (8.8177, 8.5727)
Got score: 8.4633 (8.4570, 8.4695)
Got score: 8.7016 (8.7841, 8.6191)


{'n_hidden_layers': 6, 'units_mult': 16, 'unit_decrease_factor': 2.0, 'batch_normalization': True, 'input_batch_normalization': False, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 1623361
Got score: 8.6808 (8.6562, 8.7054)
Got score: 8.7015 (8.7257, 8.6774)
Got score: 8.7397 (8.6728, 8.8067)
Got score: 8.8573 (8.7289, 8.9856)
Got score: 9.0052 (9.3475, 8.6630)


In [11]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.38,0.14,21217
1,8.33,0.23,1624705
2,8.41,0.15,67777
3,8.60,0.11,51457
4,8.80,0.12,1623361
